In [1]:
import numpy as np
import datetime
from tqdm import tqdm
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.exact import solve_tsp_brute_force
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [2]:
N = 5
nlen = N*N
rand = np.random.RandomState(1)

In [3]:
cnt = 40000
l = []
for i in tqdm(range(cnt)):
    a = np.round(rand.normal(5, 1.5, size=(N, N)),2)
    a[np.eye(N)==1] = 0
    a = (a / a.max())
    permutation, distance = solve_tsp_dynamic_programming(a)
    l.append(np.concatenate((a,permutation,distance),axis=None)) 
ds = np.array(l)

100%|██████████████████████████████████| 40000/40000 [00:02<00:00, 17082.10it/s]


In [4]:
#np.save('ds_tsp.npy', ds)
#ds = np.load('ds_tsp.npy')

In [5]:
clf = RandomForestClassifier(max_depth=20, random_state=0) 
# clf = MultiOutputClassifier(LogisticRegression())
# clf = MultiOutputClassifier(XGBClassifier())

In [6]:
clf.fit(ds[:,0:nlen], ds[:,nlen+1:nlen+N].astype(int)-1)

RandomForestClassifier(max_depth=20, random_state=0)

In [7]:
y_pred = clf.predict(ds[:,0:nlen])

In [8]:
y_pred = y_pred + 1

In [9]:
for i in range(N-1):
    print(accuracy_score(ds[:,i+nlen+1], y_pred[:,i]))

1.0
1.0
1.0
1.0


In [10]:
cnt = 1000
l = []
for i in range(cnt):
    a = np.round(rand.normal(5, 1.5, size=(N, N)),2)
    a[np.eye(N)==1] = 0
    a = (a / a.max())
    permutation, distance = solve_tsp_dynamic_programming(a)
    l.append(np.concatenate((a,permutation,distance),axis=None)) 
ds_test = np.array(l)

In [11]:
y_pred = clf.predict(ds_test[:,0:nlen])
y_pred = y_pred + 1

In [12]:
prob_test = clf.predict_proba(ds_test[:,0:nlen])

In [13]:
err = 0
for i in range(len(ds_test)):
    flag = False
    for j in range(1,N-1):
        #print(y_pred[i,j], y_pred[i,0:j])
        if y_pred[i,j] in y_pred[i,0:j]:
            flag = True 
            #print(i, j, y_pred[i], prob_test[j][i], np.flip((np.argsort(prob_test[j][i])+1)))
            for d in np.flip((np.argsort(prob_test[j][i])+1)):
                if d not in y_pred[i,0:j]:
                    y_pred[i,j] = d
                    break 
    """
    path = np.zeros(N-1)
    x = np.array([prob_test[j][i] for j in range(0,N-1)])
    for r, c in np.dstack(np.unravel_index(np.argsort(-x, axis = None), x.shape))[0]:
        if path[r] == 0 and (c+1) not in path:
            path[r] = c + 1 
        if np.all(path>0):
            break
    y_pred[i] = path"""
    if flag:
        err += 1
print(err)

483


In [14]:
for i in range(N-1):
    print(accuracy_score(ds_test[:,i+nlen+1], y_pred[:,i]))

0.707
0.62
0.573
0.664


In [15]:
dist_pred = np.zeros(cnt)
for i in range(len(ds_test)):
    a = ds_test[i,0:nlen].reshape(N,N)
    path = np.concatenate([[0], y_pred[i]])
    dist_pred[i] = sum(a[int(x),int(y)] for x,y in zip(path[:-1],path[1:]))+a[int(path[-1]),int(path[0])]
    if np.round(ds_test[i,-1],4) > np.round(dist_pred[i],4): # Проверка на магию
        print(i,ds_test[i,-1], dist_pred)

In [16]:
mean_absolute_percentage_error(ds_test[:,-1],dist_pred)

0.05892713475123422

In [17]:
ddiff = np.concatenate([np.expand_dims(ds_test[:,-1], axis=1), np.expand_dims(dist_pred[:], axis=1)],axis = 1)

In [18]:
# Сколько случаев действительно плохого прогноза
sum(((ddiff[:,1] - ddiff[:,0]) / ddiff[:,0]) > 0.1)

220